<a href="https://colab.research.google.com/github/Skuerjuan/tp2-chona/blob/main/maierowicz_atanasoff_gallo_ABD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 Seminario IA
# Maierowicz, Atanasoff, Gallo
---
## ¿Deberían aceptar darle el préstamo?
### (Esta vez con redes neuronales)

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [17]:
# Preprocesamiento

data = "https://raw.githubusercontent.com/Skuerjuan/tp1_chona/refs/heads/main/loan_data.csv"
df = pd.read_csv(data)

categorical = ["person_gender", "person_education", "person_home_ownership", "loan_intent", "previous_loan_defaults_on_file"]

ct = ColumnTransformer(transformers=[("cat", OneHotEncoder(), categorical)], remainder="passthrough")

x = ct.fit_transform(df)

feature_names = ct.get_feature_names_out()

df = pd.DataFrame(x, columns=feature_names)
df_x = df.drop(columns="remainder__loan_status")
df_y = df["remainder__loan_status"]

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [18]:
# deberia haber un hyperparam de cuantas layers hay en el medio y otro de cuanto es el n
# otro más que sea el activation sería una buena idea (?)

model = keras.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.Dense(15, activation="relu"),
    layers.Dense(2, activation="softmax")
])

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_50 (Dense)                │ (None, 15)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 2)              │            32 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 452 (1.77 KB)

 Trainable params: 452 (1.77 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7244 - loss: 274.0557
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7332 - loss: 14.6131
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7384 - loss: 10.7784
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7518 - loss: 13.5533
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7550 - loss: 12.9888
Epoch 6/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7664 - loss: 11.1872
Epoch 7/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7743 - loss: 10.6865
Epoch 8/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7830 - loss: 9.2744
Epoch 9/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7866 - loss: 9.8469
Epoch 10/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7827 - loss: 10.3636


In [20]:
score = model.evaluate(x_test, y_test, batch_size=64, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

y_pred_probs = model.predict(x_test)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

print(f'Precision: {precision_score(y_test, y_pred_labels)}\n')
print(f'Recall: {recall_score(y_test, y_pred_labels)}\n')

Test loss: 9.278127670288086
Test accuracy: 0.8295555710792542
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Precision: 0.9327272727272727

Recall: 0.25522388059701495



In [ ]:
# evaluar el mejor set de hyperparams que quede al final
# comparar con el tp anterior
from sklearn.model_selection import KFold

EPOCHS = 10
N_SPLITS = 5

def model_with_params(layers_num: int, neurons_per_layer: int, activation: str = "relu"):
  model = keras.Sequential()
  model.add(layers.Input(shape=(x_train.shape[1],)))
  for i in range(layers_num):
    model.add(layers.Dense(neurons_per_layer, activation=activation))
  model.add(layers.Dense(2, activation="softmax"))
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  return model

# Sacado de acá: https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
def kfold_validation(model_opts, kfold: KFold):
  fold_no = 1
  acc_per_fold = []
  loss_per_fold = []
  _df_x = df_x.to_numpy()
  _df_y = df_y.to_numpy()
  for train, test in kfold.split(df_x, df_y):
#    model = model_with_params(layers_num=model_opts[0], neurons_per_layer=model_opts[1], activation=model_opts[2])
    model = model_with_params(1, 1)
    print(f'Training for fold {fold_no} ...')
    model.fit(_df_x[train], _df_y[train], epochs=EPOCHS, verbose=1)
    print(f'Evaluating...')
    score = model.evaluate(_df_x[test], _df_y[test], verbose=1)
    print(f'Done...')
    acc_per_fold.append(score[1])
    loss_per_fold.append(score[0])
    fold_no += 1
  return {
      "accuracy": np.mean(acc_per_fold),
      "loss": np.mean(loss_per_fold)
  }

modelos = [
  # Probamos uno grande
  [5, 15, "relu"],
  # Probamos uno chico
  [2, 5, "relu"],
  # Los mismos pero con otra función de activación
  [5, 15, "sigmoid"],
  [2, 5, "softplus"],
]

kfold = KFold(n_splits=10, shuffle=True)

for model_params in modelos:
  kfold_validation(model_params, kfold)

Training for fold 1 ...
Epoch 1/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7740 - loss: 0.6044
Epoch 2/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7749 - loss: 0.5334
Epoch 3/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7760 - loss: 0.5320
Epoch 4/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7744 - loss: 0.5340
Epoch 5/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7780 - loss: 0.5295
Epoch 6/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7783 - loss: 0.5291
Epoch 7/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7774 - loss: 0.5302
Epoch 8/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7785 - loss: 0.5289
Epoch 9/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7764 - loss: 0.5315
Epoch 10/10
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7778 - loss: 0.5297
Evaluating...
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7569 - loss: 0.5560
Done..